# Neon Caster - Juego de Visión por Computador

Juego interactivo que usa MediaPipe Pose para detectar movimientos del cuerpo y controlar mecánicas de defensa y ataque.

## Configuración Global del Juego

Esta celda define todas las constantes y parámetros del juego: dimensiones de pantalla, configuración de MediaPipe, colores, estadísticas del jugador (vidas, escudos, balas), mecánicas de combate (escudo y ataque), comportamiento de enemigos, sistema de rondas progresivas, power-ups y rutas de assets.

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import math
import random
import time
import pygame
import os

# =============================================================================
# CONFIGURACIÓN GLOBAL DEL JUEGO
# =============================================================================

# --- Pantalla ---
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 720
FPS = 30

# --- MediaPipe ---
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# --- Colores (Formato BGR para OpenCV) ---
COLOR_NEON_BLUE = (255, 255, 0)    # Cian
COLOR_NEON_PINK = (203, 192, 255)  # Rosa/Morado
COLOR_NEON_GOLD = (0, 215, 255)    # Dorado
COLOR_RED = (0, 0, 255)
COLOR_GREEN = (0, 255, 0)
COLOR_WHITE = (255, 255, 255)

# --- Jugador ---
MAX_LIVES = 3
MAX_SHIELDS = 3
MAX_BULLETS = 3
HITBOX_RADIUS = 30  # Radio de colisión en puntos del cuerpo

# --- Escudo ---
SHIELD_ACTIVATION_DIST = 200      # Distancia máxima entre muñecas para activar escudo
SHIELD_COOLDOWN_FRAMES = 120      # Frames de cooldown (4 segundos a 30 FPS)
SHIELD_CD_TICK_INTERVAL = 30      # Frames entre cada tick de sonido
SHIELD_VISUAL_SIZE = 350          # Tamaño del sprite de escudo

# --- Ataque ---
ATTACK_COOLDOWN_FRAMES = 90       # Frames de cooldown (3 segundos a 30 FPS)
ATTACK_CHARGE_RATE = 2            # Velocidad de carga de energía
ATTACK_DISCHARGE_RATE = 5         # Velocidad de descarga
ATTACK_MIN_WRIST_RATIO = 0.5      # Ratio mínimo muñecas/hombros para cargar
ATTACK_MAX_WRIST_RATIO = 1.2      # Ratio máximo muñecas/hombros para cargar
ATTACK_TRIGGER_RATIO = 1.5        # Ratio para disparar (manos muy separadas)

# --- Enemigos ---
ENEMY_SPEED_MIN = 15            
ENEMY_SPEED_MAX = 20
ENEMY_WARNING_FRAMES = 50        # Frames de advertencia antes de aparecer
ENEMY_SPAWN_BASE_RATE = 60        # Frames entre spawns (base) - ~2 segundos
ENEMY_SPAWN_MIN_RATE = 30        # Frames mínimos entre spawns - ~1 segundo
SPAWN_PAUSE_AFTER_ATTACK = 60     # Pausa de spawn tras golpe de choque

# --- Sistema de Rondas ---
ROUND_POINTS_REQUIREMENT = 15  # Puntos necesarios para pasar de ronda
ROUND_SPEED_MULTIPLIER = 1.2     # Multiplicador de velocidad por ronda (+20% más rápido)
ROUND_SPAWN_RATE_MULTIPLIER = 0.6 # Multiplicador de spawn rate más frecuente por ronda (-50% tiempo entre spawns)

# --- Sonido/Medalla de ronda ---
ROUND_COMPLETE_SOUND_CANDIDATES = ('round', 'round_complete', 'fanfare', 'level-up', 'victory')
MEDAL_FILENAMES = ['medal1.png', 'medal2.png', 'medal3.png'] 
MEDAL_DISPLAY_TIME = 150     # Frames para mostrar la medalla (6s a 30 FPS)

# --- Power-ups ---
POWERUP_SPAWN_MIN = 300           # Frames mínimo entre power-ups (~10s)
POWERUP_SPAWN_MAX = 600           # Frames máximo entre power-ups (~30s)
POWERUP_SPEED_MIN = 2             # Velocidad mínima de caída
POWERUP_SPEED_MAX = 4             # Velocidad máxima de caída
POWERUP_CATCH_RADIUS = 60         # Radio de captura con la mano

# --- Assets (rutas) ---
BASE = os.path.abspath(".")
ASSET_ENEMIES = os.path.join(BASE, 'assets', 'enemies')
ASSET_SFX = os.path.join(BASE, 'assets', 'sfx')
ASSET_HUD = os.path.join(BASE, 'assets', 'hud')
ASSET_HITS = os.path.join(BASE, 'assets', 'hits')

print("Configuración global cargada correctamente")
print(f"   Pantalla: {SCREEN_WIDTH}x{SCREEN_HEIGHT} @ {FPS} FPS")
print(f"   Vidas: {MAX_LIVES} | Escudos: {MAX_SHIELDS} | Balas: {MAX_BULLETS}")
print(f"   Sistema de Rondas: {ROUND_POINTS_REQUIREMENT} puntos por ronda")

Configuración global cargada correctamente
   Pantalla: 1280x720 @ 30 FPS
   Vidas: 3 | Escudos: 3 | Balas: 3
   Sistema de Rondas: 15 puntos por ronda


## Funciones de Utilidad y Carga de Assets

Contiene funciones auxiliares para cargar imágenes y sonidos con fallbacks de seguridad, buscar archivos de audio por prefijo, calcular distancias euclidianas entre puntos, y generar el spritesheet de explosiones de forma procedural si no existe.

In [7]:
# =============================================================================
# FUNCIONES DE UTILIDAD - CARGA DE ASSETS
# =============================================================================

def load_image(path, size=None):
    """Carga una imagen desde disco con fallback si no existe."""
    if not os.path.exists(path):
        print(f'Imagen no encontrada: {path}')
        w, h = size if size else (40, 40)
        s = pygame.Surface((w, h), pygame.SRCALPHA)
        pygame.draw.circle(s, (255, 0, 0), (w//2, h//2), min(w, h)//2)
        return s
    try:
        surf = pygame.image.load(path).convert_alpha()
        if size:
            surf = pygame.transform.smoothscale(surf, size)
        return surf
    except Exception as e:
        print(f'Error cargando imagen {path}: {e}')
        w, h = size if size else (40, 40)
        s = pygame.Surface((w, h), pygame.SRCALPHA)
        pygame.draw.circle(s, (255, 0, 0), (w//2, h//2), min(w, h)//2)
        return s


def find_sound_file(prefix):
    """Busca un archivo de sonido por prefijo en la carpeta de SFX."""
    if not os.path.exists(ASSET_SFX):
        return None
    audio_ext = ('.wav', '.ogg', '.mp3')
    
    # Buscar por prefijo exacto
    for f in os.listdir(ASSET_SFX):
        lname = f.lower()
        if lname.startswith(prefix.lower()) and lname.endswith(audio_ext):
            return os.path.join(ASSET_SFX, f)
    
    # Buscar si contiene el prefijo
    for f in os.listdir(ASSET_SFX):
        lname = f.lower()
        if prefix.lower() in lname and lname.endswith(audio_ext):
            return os.path.join(ASSET_SFX, f)
    return None


def load_sound(path):
    """Carga un sonido desde disco."""
    if not path or not os.path.exists(path):
        return None
    try:
        return pygame.mixer.Sound(path)
    except Exception as e:
        print(f'Error cargando sonido {path}: {e}')
        return None


def calculate_distance(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos."""
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])


def generate_explosion_spritesheet():
    """Genera el spritesheet de explosión si no existe."""
    explosion_path = os.path.join(ASSET_HITS, 'explosion.png')
    
    if not os.path.exists(ASSET_HITS):
        os.makedirs(ASSET_HITS)
        print(f"Carpeta 'hits' creada")
    
    if os.path.exists(explosion_path):
        print(f"Spritesheet de explosión existe")
        return
    
    print("Generando spritesheet de explosión...")
    frame_size = 120
    spritesheet = pygame.Surface((frame_size * 6, frame_size), pygame.SRCALPHA)
    
    colors = [
        (255, 50, 0, 200), (255, 100, 0, 220), (255, 150, 0, 230),
        (255, 200, 0, 240), (255, 255, 100, 200), (200, 150, 50, 100)
    ]
    
    for i, color in enumerate(colors):
        cx = i * frame_size + frame_size // 2
        cy = frame_size // 2
        pygame.draw.circle(spritesheet, color, (cx, cy), int(frame_size * 0.35))
        for ring in range(2):
            r = int(frame_size * (0.25 + ring * 0.15))
            pygame.draw.circle(spritesheet, (*color[:3], max(0, color[3] - ring * 50)), (cx, cy), r, 3)
        random.seed(i)
        for _ in range(5 + i):
            sx = cx + random.randint(-frame_size//3, frame_size//3)
            sy = cy + random.randint(-frame_size//3, frame_size//3)
            pygame.draw.circle(spritesheet, (255, 255, 100, max(0, 150 - i * 20)), (sx, sy), 2)
    
    pygame.image.save(spritesheet, explosion_path)
    print(f"Spritesheet generado: {explosion_path}")


print("Funciones de utilidad cargadas")


Funciones de utilidad cargadas


## Clases del Juego: Enemy y PowerUp

Define las clases principales del juego:
- **Enemy**: Enemigos que aparecen desde los bordes de la pantalla con fase de advertencia, animación de impacto y movimiento hacia el lado opuesto.
- **PowerUp**: Objetos que caen desde arriba y pueden ser capturados con las manos para recuperar vidas, escudos o balas.

In [8]:
# =============================================================================
# CLASES DEL JUEGO
# =============================================================================

class Enemy:
    """Enemigo que aparece desde los bordes y atraviesa la pantalla."""
    
    def __init__(self, screen_w, screen_h, img, hit_frames, hit_sound):
        self.screen_w, self.screen_h = screen_w, screen_h
        self.image = img
        self.hit_frames = hit_frames or []
        self.hit_sound = hit_sound
        self.radius = 15
        self.active = True
        self.hit = False
        self.collision_processed = False
        self.warning_time = ENEMY_WARNING_FRAMES
        self.hit_anim_t = 0
        self.hit_frame_idx = 0
        
        # Posición inicial y objetivo (desde/hacia un borde aleatorio)
        self._init_position()
        
        # Velocidad
        dx, dy = self.target_x - self.x, self.target_y - self.y
        dist = math.sqrt(dx**2 + dy**2) or 1
        self.speed = random.uniform(ENEMY_SPEED_MIN, ENEMY_SPEED_MAX)
        self.vx = (dx / dist) * self.speed
        self.vy = (dy / dist) * self.speed
        
        # Tipo aleatorio
        self.type = random.choice(["DODGE", "ATTACK"])
    
    def _init_position(self):
        """Inicializa posición desde un borde aleatorio."""
        side = random.randint(0, 3)
        margin = 50
        if side == 0:  # Arriba
            self.x = random.randint(0, self.screen_w)
            self.y = -margin
            self.target_x = random.randint(0, self.screen_w)
            self.target_y = self.screen_h + margin
        elif side == 1:  # Derecha
            self.x = self.screen_w + margin
            self.y = random.randint(0, self.screen_h)
            self.target_x = -margin
            self.target_y = random.randint(0, self.screen_h)
        elif side == 2:  # Abajo
            self.x = random.randint(0, self.screen_w)
            self.y = self.screen_h + margin
            self.target_x = random.randint(0, self.screen_w)
            self.target_y = -margin
        else:  # Izquierda
            self.x = -margin
            self.y = random.randint(0, self.screen_h)
            self.target_x = self.screen_w + margin
            self.target_y = random.randint(0, self.screen_h)
    
    def move(self):
        if self.active and not self.hit:
            if self.warning_time > 0:
                self.warning_time -= 1
            else:
                self.x += self.vx
                self.y += self.vy
    
    def take_hit(self):
        if self.hit:
            return
        self.hit = True
        self.hit_anim_t = 0
        self.hit_frame_idx = 0
        if self.hit_sound:
            try:
                self.hit_sound.play()
            except:
                pass
    
    def is_off_screen(self):
        margin = 100
        return (self.x < -margin or self.x > self.screen_w + margin or 
                self.y < -margin or self.y > self.screen_h + margin)
    
    def draw(self, surface):
        if not self.active:
            return
        
        x, y = int(self.x), int(self.y)
        
        # Fase de advertencia
        if self.warning_time > 0 and not self.hit:
            entry = self._get_entry_point()
            ex, ey = int(entry[0]), int(entry[1])
            alpha = 128 + int(127 * math.sin(time.time() * 10))
            
            # Círculo pulsante (más grande y visible)
            circle_size = 60
            s = pygame.Surface((circle_size, circle_size), pygame.SRCALPHA)
            pygame.draw.circle(s, (255, 165, 0, alpha), (circle_size//2, circle_size//2), 20)
            pygame.draw.circle(s, (255, 100, 0, alpha), (circle_size//2, circle_size//2), 20, 3)
            surface.blit(s, (ex - circle_size//2, ey - circle_size//2))
            
            # Dibujar flecha indicando dirección
            angle = math.atan2(self.vy, self.vx)
            arrow_length = 90
            arrow_width = 30
            
            # Calcular puntos de la flecha
            end_x = ex + math.cos(angle) * arrow_length
            end_y = ey + math.sin(angle) * arrow_length
            
            # Puntas de la flecha
            arrow_angle1 = angle + math.pi * 0.75
            arrow_angle2 = angle - math.pi * 0.75
            tip1_x = end_x + math.cos(arrow_angle1) * arrow_width
            tip1_y = end_y + math.sin(arrow_angle1) * arrow_width
            tip2_x = end_x + math.cos(arrow_angle2) * arrow_width
            tip2_y = end_y + math.sin(arrow_angle2) * arrow_width
            
            # Dibujar línea principal de la flecha
            pygame.draw.line(surface, (255, 200, 0, 255), (ex, ey), (int(end_x), int(end_y)), 5)
            
            # Dibujar punta de flecha
            pygame.draw.polygon(surface, (255, 200, 0, 255), [
                (int(end_x), int(end_y)),
                (int(tip1_x), int(tip1_y)),
                (int(tip2_x), int(tip2_y))
            ])
            
            return
        
        # Animación de golpe
        if self.hit:
            if self.hit_frames:
                frame = self.hit_frames[self.hit_frame_idx]
                rect = frame.get_rect(center=(x, y))
                surface.blit(frame, rect)
                self.hit_anim_t += 1
                if self.hit_anim_t % 4 == 0:
                    self.hit_frame_idx += 1
                    if self.hit_frame_idx >= len(self.hit_frames):
                        self.active = False
            else:
                t = self.hit_anim_t
                r = min(60, 10 + t * 4)
                alpha = max(0, 200 - t * 12)
                s = pygame.Surface((r*2, r*2), pygame.SRCALPHA)
                pygame.draw.circle(s, (255, 200, 0, alpha), (r, r), r)
                surface.blit(s, (x - r, y - r))
                self.hit_anim_t += 1
                if self.hit_anim_t > 10:
                    self.active = False
            return
        
        # Dibujo normal
        angle = math.degrees(math.atan2(self.vy, self.vx))
        rotated = pygame.transform.rotozoom(self.image, -angle, 1.0)
        rect = rotated.get_rect(center=(x, y))
        surface.blit(rotated, rect)
    
    def _get_entry_point(self):
        """Calcula el punto de entrada visible en pantalla."""
        entry_x = max(0, min(self.screen_w, self.x))
        entry_y = max(0, min(self.screen_h, self.y))
        
        if self.x < 0 and self.vx != 0:
            t = -self.x / self.vx
            entry_x, entry_y = 0, self.y + self.vy * t
        elif self.x > self.screen_w and self.vx != 0:
            t = (self.screen_w - self.x) / self.vx
            entry_x, entry_y = self.screen_w, self.y + self.vy * t
        
        if self.y < 0 and self.vy != 0:
            t = -self.y / self.vy
            entry_x, entry_y = self.x + self.vx * t, 0
        elif self.y > self.screen_h and self.vy != 0:
            t = (self.screen_h - self.y) / self.vy
            entry_x, entry_y = self.x + self.vx * t, self.screen_h
        
        return entry_x, entry_y


class PowerUp:
    """Power-up que cae desde arriba y otorga vida, escudos o balas."""
    
    def __init__(self, screen_w, screen_h, powerup_type, img):
        self.screen_w, self.screen_h = screen_w, screen_h
        self.type = powerup_type  # 'life', 'shield', 'bullet'
        self.image = img
        self.x = random.randint(100, screen_w - 100)
        self.y = -50
        self.speed = random.uniform(POWERUP_SPEED_MIN, POWERUP_SPEED_MAX)
        self.active = True
        self.collected = False
        self.collect_anim_t = 0
    
    def move(self):
        if self.active and not self.collected:
            self.y += self.speed
    
    def is_off_screen(self):
        return self.y > self.screen_h + 100
    
    def check_hand_catch(self, left_hand, right_hand):
        """Verifica si alguna mano está cerca del power-up."""
        if self.collected:
            return False
        dist_left = calculate_distance((self.x, self.y), left_hand)
        dist_right = calculate_distance((self.x, self.y), right_hand)
        return dist_left < POWERUP_CATCH_RADIUS or dist_right < POWERUP_CATCH_RADIUS
    
    def collect(self):
        if not self.collected:
            self.collected = True
            self.collect_anim_t = 0
    
    def draw(self, surface):
        if not self.active:
            return
        
        x, y = int(self.x), int(self.y)
        
        if self.collected:
            # Animación de recolección
            t = self.collect_anim_t
            scale = 1.0 + t * 0.3
            alpha = max(0, 255 - t * 25)
            w, h = self.image.get_size()
            scaled = pygame.transform.smoothscale(self.image, (int(w * scale), int(h * scale)))
            scaled.set_alpha(alpha)
            rect = scaled.get_rect(center=(x, y))
            surface.blit(scaled, rect)
            pygame.draw.circle(surface, (255, 255, 100), (x, y), int(30 * scale), 3)
            self.collect_anim_t += 1
            if self.collect_anim_t > 10:
                self.active = False
        else:
            # Dibujo normal con efecto pulsante
            pulse = math.sin(time.time() * 5) * 0.1 + 1.0
            w, h = self.image.get_size()
            pulsed = pygame.transform.smoothscale(self.image, (int(w * pulse), int(h * pulse)))
            rect = pulsed.get_rect(center=(x, y))
            surface.blit(pulsed, rect)
            
            # Aura
            aura_radius = int(35 * pulse)
            aura_color = {'life': (255, 100, 100), 'shield': (100, 200, 255), 'bullet': (255, 200, 50)}.get(self.type, (255, 255, 255))
            aura_surf = pygame.Surface((aura_radius*2, aura_radius*2), pygame.SRCALPHA)
            pygame.draw.circle(aura_surf, (*aura_color, 80), (aura_radius, aura_radius), aura_radius)
            surface.blit(aura_surf, (x - aura_radius, y - aura_radius))


print("Clases Enemy y PowerUp cargadas")


Clases Enemy y PowerUp cargadas


## Funciones de Colisión y Efectos Visuales

Implementa la detección de colisiones entre enemigos y puntos del esqueleto del jugador, el dibujado de zonas de hitbox translúcidas sobre el cuerpo, efectos visuales de carga de ataque con partículas, explosiones de área y pantallas de completado de ronda.

In [9]:
# =============================================================================
# FUNCIONES DE COLISIÓN Y VISUALES
# =============================================================================

# Puntos clave del cuerpo para detección de colisiones
BODY_KEYPOINTS = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]


def check_skeleton_collision(enemy, landmarks, w, h):
    """Verifica colisión entre un enemigo y los puntos del esqueleto."""
    for idx in BODY_KEYPOINTS:
        px = int(landmarks[idx].x * w)
        py = int(landmarks[idx].y * h)
        if calculate_distance((enemy.x, enemy.y), (px, py)) < enemy.radius + HITBOX_RADIUS:
            return True, (px, py)
    return False, None


def draw_hitbox_zones(image, landmarks, w, h, shield_active=False):
    """Dibuja las zonas de colisión visibles sobre el jugador."""
    circle_color = COLOR_NEON_GOLD if shield_active else COLOR_NEON_BLUE
    fill_color = (0, 215, 255) if shield_active else (255, 255, 0)
    
    for idx in BODY_KEYPOINTS:
        px = int(landmarks[idx].x * w)
        py = int(landmarks[idx].y * h)
        
        # Círculo translúcido
        overlay = image.copy()
        cv2.circle(overlay, (px, py), HITBOX_RADIUS, fill_color, -1)
        cv2.addWeighted(overlay, 0.3, image, 0.7, 0, image)
        
        # Borde y centro
        cv2.circle(image, (px, py), HITBOX_RADIUS, circle_color, 2)
        cv2.circle(image, (px, py), max(2, HITBOX_RADIUS // 7), (255, 255, 255), -1)


def draw_charge_visual(screen, mid_x, mid_y, energy_charge):
    """Dibuja el visual de carga de ataque."""
    if energy_charge >= 100:
        # Carga completa - visual dorado pulsante
        pulse = int(10 * abs(math.sin(time.time() * 10)))
        radius = 50 + pulse
        pygame.draw.circle(screen, (0, 215, 255), (mid_x, mid_y), radius)
        pygame.draw.circle(screen, COLOR_WHITE, (mid_x, mid_y), radius + 5, 3)
        
        # Partículas giratorias
        for angle in range(0, 360, 30):
            rad = math.radians(angle + time.time() * 200)
            px = int(mid_x + math.cos(rad) * (radius + 20))
            py = int(mid_y + math.sin(rad) * (radius + 20))
            pygame.draw.circle(screen, COLOR_WHITE, (px, py), 5)
        
        # Textos
        ready_font = pygame.font.SysFont('Arial', 40, bold=True)
        inst_font = pygame.font.SysFont('Arial', 20)
        ready_surf = ready_font.render("¡LISTO!", True, COLOR_WHITE)
        inst_surf = inst_font.render("SEPARA LAS MANOS", True, (0, 215, 255))
        screen.blit(ready_surf, (mid_x - ready_surf.get_width()//2, mid_y - 70))
        screen.blit(inst_surf, (mid_x - inst_surf.get_width()//2, mid_y + 70))
    else:
        # Cargando progresivamente
        intensity = int(energy_charge * 2.55)
        radius = int(20 + (energy_charge / 100) * 30)
        pygame.draw.circle(screen, (0, intensity, 255), (mid_x, mid_y), radius)
        pygame.draw.circle(screen, COLOR_WHITE, (mid_x, mid_y), radius, 2)
        
        # Barra de progreso
        bar_w = 120
        bar_x = mid_x - bar_w // 2
        bar_y = mid_y - 60
        pygame.draw.rect(screen, (50, 50, 50), (bar_x, bar_y, bar_w, 12))
        pygame.draw.rect(screen, (0, intensity, 255), (bar_x, bar_y, int(bar_w * energy_charge / 100), 12))
        pygame.draw.rect(screen, COLOR_WHITE, (bar_x, bar_y, bar_w, 12), 2)


def draw_explosion_effect(screen, w, h, effect_progress, max_duration=20):
    """Dibuja el efecto de explosión tras un golpe de choque."""
    alpha = int(200 * (effect_progress / max_duration))
    
    # Flash blanco
    flash = pygame.Surface((w, h))
    flash.fill(COLOR_WHITE)
    flash.set_alpha(alpha)
    screen.blit(flash, (0, 0))
    
    # Ondas expansivas
    wave_radius = int(300 * (1 - effect_progress / max_duration))
    cx, cy = w // 2, h // 2
    for i in range(3):
        pygame.draw.circle(screen, (255, 200, 0), (cx, cy), wave_radius + i * 50, 5)


def draw_round_complete_screen(screen, screen_w, screen_h, round_num, score, points_next):
    """Dibuja la pantalla de ronda completada."""
    # Fondo semi-transparente
    overlay = pygame.Surface((screen_w, screen_h), pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 180))
    screen.blit(overlay, (0, 0))
    
    # Texto de ronda completada
    big_font = pygame.font.SysFont('Arial', 80, bold=True)
    small_font = pygame.font.SysFont('Arial', 36)
    
    round_text = big_font.render(f"¡RONDA {round_num} COMPLETADA!", True, (0, 215, 255))
    score_text = small_font.render(f"Score: {score}", True, COLOR_WHITE)
    next_text = small_font.render(f"Puntos para siguiente ronda: {points_next}", True, (255, 200, 0))
    
    # Centrar textos
    screen.blit(round_text, ((screen_w - round_text.get_width()) // 2, screen_h // 2 - 100))
    screen.blit(score_text, ((screen_w - score_text.get_width()) // 2, screen_h // 2 - 20))
    screen.blit(next_text, ((screen_w - next_text.get_width()) // 2, screen_h // 2 + 50))


print("Funciones de colisión y visuales cargadas")


Funciones de colisión y visuales cargadas


## Bucle Principal del Juego

Función principal que inicializa Pygame, carga todos los assets (imágenes, sonidos, sprites), configura la cámara y MediaPipe Pose, y ejecuta el bucle de juego completo que procesa la detección de poses, gestiona enemigos y power-ups, detecta colisiones, controla las mecánicas de escudo y ataque, dibuja el HUD y maneja el sistema de rondas progresivas.

In [ ]:
# =============================================================================
# BUCLE PRINCIPAL DEL JUEGO
# =============================================================================

def start_game():
    """Inicializa y ejecuta el bucle principal del juego."""
    print("\n" + "="*60)
    print("INICIANDO NEON CASTER...")
    print("="*60 + "\n")
    
    # Inicializar Pygame
    pygame.init()
    try:
        pygame.mixer.init()
    except:
        pass
    
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Neon Caster")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont('Arial', 28)
    
    # Generar assets si no existen
    generate_explosion_spritesheet()
    
    # =========================================================================
    # CARGAR ASSETS
    # =========================================================================
    print("Cargando assets...")
    
    # Imágenes de enemigos
    enemy_images = []
    if os.path.exists(ASSET_ENEMIES):
        for f in os.listdir(ASSET_ENEMIES):
            if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                enemy_images.append(load_image(os.path.join(ASSET_ENEMIES, f), size=(60, 60)))
    print(f"   {len(enemy_images)} enemigos")
    
    # Frames de explosión
    hit_frames = []
    spritesheet_path = os.path.join(ASSET_HITS, 'explosion.png')
    if os.path.exists(spritesheet_path):
        try:
            sheet = pygame.image.load(spritesheet_path).convert_alpha()
            fw = sheet.get_width() // 6
            fh = sheet.get_height()
            for i in range(6):
                frame = sheet.subsurface(pygame.Rect(i * fw, 0, fw, fh)).copy()
                hit_frames.append(pygame.transform.smoothscale(frame, (60, 60)))
        except Exception as e:
            print(f"   Error cargando explosión: {e}")
    print(f"   {len(hit_frames)} frames de explosión")
    
    # Sonidos
    sfx_hit = load_sound(find_sound_file('hit'))
    sfx_block = load_sound(find_sound_file('shield'))
    sfx_tick = load_sound(find_sound_file('tick') or find_sound_file('beep'))
    soundtrack = load_sound(find_sound_file('pixel'))
    sfx_explosion = load_sound(find_sound_file('punch'))
    sfx_game_over = load_sound(find_sound_file('game_over'))
    sfx_powerup = load_sound(find_sound_file('powerup'))
    round_sounds = []
    for name in ROUND_COMPLETE_SOUND_CANDIDATES:
        path = find_sound_file(name)
        s = load_sound(path)
        if s:
            round_sounds.append(s)

   
    print(f"   Sonidos cargados")
    
    # HUD
    img_heart_full = load_image(os.path.join(ASSET_HUD, 'heart.png'), (36, 36))
    img_heart_empty = load_image(os.path.join(ASSET_HUD, 'heart2.png'), (36, 36))
    img_shield_full = load_image(os.path.join(ASSET_HUD, 'shield2.png'), (36, 36))
    img_shield_empty = load_image(os.path.join(ASSET_HUD, 'shield3.png'), (36, 36))
    img_bullet_full = load_image(os.path.join(ASSET_HUD, 'bullet.png'), (36, 36))
    img_bullet_empty = load_image(os.path.join(ASSET_HUD, 'bullet2.png'), (36, 36))
   

    img_medals = []
    for fname in MEDAL_FILENAMES:
        img_medals.append(
            load_image(os.path.join(ASSET_HUD, fname), (200, 200))
        )
        
   

    # Power-ups
    img_powerup = {
        'life': load_image(os.path.join(ASSET_HUD, 'heart.png'), (50, 50)),
        'shield': load_image(os.path.join(ASSET_HUD, 'shield2.png'), (50, 50)),
        'bullet': load_image(os.path.join(ASSET_HUD, 'bullet.png'), (50, 50))
    }
    
    # Escudo visual
    shield_path = os.path.join(ASSET_HUD, 'shield.png')
    try:
        img_shield_icon = pygame.image.load(shield_path).convert_alpha() if os.path.exists(shield_path) else load_image(shield_path, (96, 96))
    except:
        img_shield_icon = load_image(shield_path, (96, 96))

 

    print("Assets cargados\n")
    
    # =========================================================================
    # ESTADO DEL JUEGO
    # =========================================================================
    state = {
        'enemies': [],
        'powerups': [],
        'score': 0,
        'lives': MAX_LIVES,
        'shields': MAX_SHIELDS,
        'bullets': MAX_BULLETS,
        'game_over': False,
        'energy_charge': 0,
        'shield_active': False,
        'shield_cooldown': 0,
        'attack_cooldown': 0,
        'spawn_timer': 0,
        'spawn_pause': 0,
        'powerup_timer': 0,
        'powerup_rate': random.randint(POWERUP_SPAWN_MIN, POWERUP_SPAWN_MAX),
        'explosion_effect': 0,
        'cd_tick_timer': 0,
        'cd_channel': None,
        
        # Rondas
        'current_round': 1,
        'current_medal_index': 0,
        'points_for_this_round': 0,
        'points_needed_for_next_round': ROUND_POINTS_REQUIREMENT,
        'medal_timer': 0,
        'round_sounds': round_sounds,
        'round_sound_channel': None
    }
    
    # Cámara y MediaPipe
    cap = cv2.VideoCapture(0)
    cap.set(3, SCREEN_WIDTH)
    cap.set(4, SCREEN_HEIGHT)
    
    if soundtrack:
        soundtrack.play(loops=-1)
    
    # =========================================================================
    # BUCLE PRINCIPAL
    # =========================================================================
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        running = True
        
        while running:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Procesar pose
            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb.flags.writeable = False
            results = pose.process(rgb)
            rgb.flags.writeable = True
            h, w = frame.shape[:2]
            
            # Dibujar hitboxes en el frame
            if results.pose_landmarks:
                draw_hitbox_zones(frame, results.pose_landmarks.landmark, w, h, state['shield_active'])
            
            # Convertir frame a surface
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            surf = pygame.surfarray.make_surface(frame_rgb.swapaxes(0, 1))
            screen.blit(surf, (0, 0))
            
            # =================================================================
            # LÓGICA DEL JUEGO
            # =================================================================
            # Solo procesar el juego si NO está mostrando la medalla de paso de ronda
            if not state['game_over'] and state['medal_timer'] <= 0:
                # Reducir cooldowns
                if state['shield_cooldown'] > 0:
                    state['shield_cooldown'] -= 1
                    state['cd_tick_timer'] += 1
                    if sfx_tick and state['cd_tick_timer'] % SHIELD_CD_TICK_INTERVAL == 0:
                        try:
                            if state['cd_channel'] and state['cd_channel'].get_busy():
                                state['cd_channel'].stop()
                            state['cd_channel'] = sfx_tick.play()
                        except:
                            pass
                    if state['shield_cooldown'] <= 0:
                        if state['cd_channel'] and state['cd_channel'].get_busy():
                            state['cd_channel'].stop()
                        state['cd_tick_timer'] = 0
                
                if state['attack_cooldown'] > 0:
                    state['attack_cooldown'] -= 1
                if state['spawn_pause'] > 0:
                    state['spawn_pause'] -= 1
                
                # Spawn de enemigos
                state['spawn_timer'] += 1
                spawn_rate = max(ENEMY_SPAWN_MIN_RATE, ENEMY_SPAWN_BASE_RATE - (state['score'] // 10))
                round_spawn_multiplier = ROUND_SPAWN_RATE_MULTIPLIER ** (state['current_round'] - 1)
                adjusted_spawn_rate = max(20, int(spawn_rate * round_spawn_multiplier))
                if state['spawn_timer'] > adjusted_spawn_rate and state['spawn_pause'] == 0 and enemy_images:
                    state['spawn_timer'] = 0
                    new_enemy = Enemy(w, h, random.choice(enemy_images), hit_frames, sfx_hit)
                    speed_multiplier = ROUND_SPEED_MULTIPLIER ** (state['current_round'] - 1)
                    new_enemy.speed *= speed_multiplier
                    new_enemy.vx *= speed_multiplier
                    new_enemy.vy *= speed_multiplier
                    state['enemies'].append(new_enemy)
                
                # Spawn de power-ups
                state['powerup_timer'] += 1
                if state['powerup_timer'] > state['powerup_rate']:
                    # Solo spawnear si hay algo que necesitemos
                    types = []
                    if state['lives'] < MAX_LIVES: types.append('life')
                    if state['shields'] < MAX_SHIELDS: types.append('shield')
                    if state['bullets'] < MAX_BULLETS: types.append('bullet')
                    
                    # Solo crear power-up si hay algún recurso que necesitemos
                    if types:
                        ptype = random.choice(types)
                        state['powerups'].append(PowerUp(w, h, ptype, img_powerup[ptype]))
                    
                    state['powerup_timer'] = 0
                    state['powerup_rate'] = random.randint(POWERUP_SPAWN_MIN, POWERUP_SPAWN_MAX)
                
                # Procesar pose del jugador
                if results.pose_landmarks:
                    lm = results.pose_landmarks.landmark
                    
                    # Extraer puntos clave
                    left_wrist = (int(lm[15].x * w), int(lm[15].y * h))
                    right_wrist = (int(lm[16].x * w), int(lm[16].y * h))
                    left_shoulder = (int(lm[11].x * w), int(lm[11].y * h))
                    right_shoulder = (int(lm[12].x * w), int(lm[12].y * h))
                    left_hip = (int(lm[23].x * w), int(lm[23].y * h))
                    right_hip = (int(lm[24].x * w), int(lm[24].y * h))
                    
                    # Cálculos de distancia
                    shoulder_width = calculate_distance(left_shoulder, right_shoulder)
                    wrist_dist = calculate_distance(left_wrist, right_wrist)
                    wrist_ratio = wrist_dist / shoulder_width if shoulder_width > 0 else 0
                    hip_y = (left_hip[1] + right_hip[1]) / 2
                    wrists_above_hips = left_wrist[1] < hip_y and right_wrist[1] < hip_y
                    
                    # Detectar brazos cruzados
                    d_rw_ls = calculate_distance(right_wrist, left_shoulder)
                    d_lw_ls = calculate_distance(left_wrist, left_shoulder)
                    d_lw_rs = calculate_distance(left_wrist, right_shoulder)
                    d_rw_rs = calculate_distance(right_wrist, right_shoulder)
                    arms_crossed = d_rw_ls < d_lw_ls and d_lw_rs < d_rw_rs
                    
                    mid_x = (left_wrist[0] + right_wrist[0]) // 2
                    mid_y = (left_wrist[1] + right_wrist[1]) // 2
                    
                    # ESCUDO: Brazos cruzados
                    if arms_crossed and wrist_dist < SHIELD_ACTIVATION_DIST and state['shield_cooldown'] == 0 and state['shields'] > 0 and wrists_above_hips:
                        state['shield_active'] = True
                        sized = pygame.transform.smoothscale(img_shield_icon, (SHIELD_VISUAL_SIZE, SHIELD_VISUAL_SIZE))
                        rect = sized.get_rect(center=(mid_x, mid_y))
                        screen.blit(sized, rect)
                    else:
                        state['shield_active'] = False
                    
                    # CARGA DE ATAQUE: Manos juntas frontalmente
                    if not arms_crossed and ATTACK_MIN_WRIST_RATIO < wrist_ratio < ATTACK_MAX_WRIST_RATIO and state['attack_cooldown'] == 0 and wrists_above_hips:
                        state['energy_charge'] = min(state['energy_charge'] + ATTACK_CHARGE_RATE, 100)
                    
                    # Visual de carga
                    if state['energy_charge'] > 0 and not arms_crossed and wrists_above_hips:
                        draw_charge_visual(screen, mid_x, mid_y, state['energy_charge'])
                    
                    # DISPARO: Separar manos
                    if not arms_crossed and wrist_ratio > ATTACK_TRIGGER_RATIO and state['energy_charge'] >= 100 and state['bullets'] > 0 and state['attack_cooldown'] == 0 and wrists_above_hips:
                        # Destruir todos los enemigos
                        destroyed = 0
                        for en in state['enemies']:
                            if en.active and en.warning_time <= 0:
                                en.active = False
                                destroyed += 1
                                state['score'] += 10
                                state['points_for_this_round'] += 10
                        if destroyed > 0 and sfx_explosion:
                            try:
                                sfx_explosion.play()
                            except:
                                pass
                        state['bullets'] -= 1
                        state['energy_charge'] = 0
                        state['attack_cooldown'] = ATTACK_COOLDOWN_FRAMES
                        state['spawn_pause'] = SPAWN_PAUSE_AFTER_ATTACK
                        state['explosion_effect'] = 20
                    elif not arms_crossed and not wrists_above_hips:
                        state['energy_charge'] = max(0, state['energy_charge'] - 3)
                    elif not arms_crossed and wrists_above_hips and state['energy_charge'] < 100 and (wrist_ratio < ATTACK_MIN_WRIST_RATIO or wrist_ratio > ATTACK_MAX_WRIST_RATIO):
                        state['energy_charge'] = max(0, state['energy_charge'] - ATTACK_DISCHARGE_RATE)
                    
                    # Efecto de explosión
                    if state['explosion_effect'] > 0:
                        draw_explosion_effect(screen, w, h, state['explosion_effect'])
                        state['explosion_effect'] -= 1
                        
                    
                    # Gestión de enemigos
                    for en in state['enemies']:
                        if en.active:
                            en.move()
                            en.draw(screen)
                            if en.warning_time <= 0:
                                collision, point = check_skeleton_collision(en, lm, w, h)
                                if collision:
                                    if state['shield_active']:
                                        if sfx_block: sfx_block.play()
                                        state['score'] += 5
                                        state['points_for_this_round'] += 5
                                        en.active = False
                                        state['shields'] -= 1
                                        state['shield_cooldown'] = SHIELD_COOLDOWN_FRAMES
                                        state['shield_active'] = False
                                    elif not en.collision_processed:
                                        en.take_hit()
                                        en.collision_processed = True
                                        state['lives'] -= 1
                                        if point:
                                            pygame.draw.circle(screen, (255, 0, 0), point, 30, 4)
                                        if state['lives'] <= 0:
                                            state['game_over'] = True
                                            if sfx_game_over:
                                                try:
                                                    sfx_game_over.play()
                                                except:
                                                    pass
                                if en.is_off_screen():
                                    en.active = False
                                    state['score'] += 2
                                    state['points_for_this_round'] += 2
                    
                    state['enemies'] = [e for e in state['enemies'] if e.active]
                    
                    # Gestión de power-ups
                    for pwup in state['powerups']:
                        if pwup.active:
                            pwup.move()
                            pwup.draw(screen)
                            if not pwup.collected and pwup.check_hand_catch(left_wrist, right_wrist):
                                pwup.collect()
                                sfx_powerup.play()
                                if pwup.type == 'life' and state['lives'] < MAX_LIVES:
                                    state['lives'] += 1
                                elif pwup.type == 'shield' and state['shields'] < MAX_SHIELDS:
                                    state['shields'] += 1
                                elif pwup.type == 'bullet' and state['bullets'] < MAX_BULLETS:
                                    state['bullets'] += 1
                                state['score'] += 3
                                state['points_for_this_round'] += 3
                            if pwup.is_off_screen():
                                pwup.active = False
                    
                    state['powerups'] = [p for p in state['powerups'] if p.active]

                # === VERIFICAR SI PASAMOS DE RONDA ===
                if state['points_for_this_round'] >= state['points_needed_for_next_round']:
                    state['current_round'] += 1
                    state['points_for_this_round'] = 0
                    state['points_needed_for_next_round'] = ROUND_POINTS_REQUIREMENT * state['current_round']
                    state['medal_timer'] = MEDAL_DISPLAY_TIME
                    if img_medals:
                            # 1–2 bronce, 3–4 plata, 5–6 oro, 7+ platino, por ejemplo
                            r = state['current_round']
                            if r <= 2:
                                idx = 0
                            elif r <= 4:
                                idx = 1
                            elif r <= 6:
                                idx = 2
                            else:
                                idx = 3
                            state['current_medal_index'] = min(idx, len(img_medals) - 1)

                    # reproducir sonido de ronda completada
                    sfx_round_channel = None
                    if state['round_sounds']:
                        try:
                            sfx = random.choice(state['round_sounds'])
                            sfx_round_channel = sfx.play()
                        except:
                            pass
                    state['round_sound_channel'] = sfx_round_channel


            # =================================================================
            # HUD
            # =================================================================
           
            
            # Score
            screen.blit(font.render(f"SCORE: {state['score']}", True, (255, 0, 0)), (550, 20))
            
            # Mostrar ronda
            round_font = pygame.font.SysFont('Arial', 32, bold=True)
            round_surf = round_font.render(f"RONDA {state['current_round']}", True, (0, 0, 0))
            screen.blit(round_surf, (SCREEN_WIDTH - round_surf.get_width() - 550, 680))
            
            # Barra de progreso de ronda
            bar_x, bar_y = 10, 700
            bar_w, bar_h = 300, 20
            progress = state['points_for_this_round'] / max(1, state['points_needed_for_next_round'])
            pygame.draw.rect(screen, (50, 50, 50), (bar_x, bar_y, bar_w, bar_h))
            pygame.draw.rect(screen, (0, 215, 255), (bar_x, bar_y, int(bar_w * progress), bar_h))
            pygame.draw.rect(screen, (255, 255, 255), (bar_x, bar_y, bar_w, bar_h), 2)
            progress_text = font.render(f"{state['points_for_this_round']}/{state['points_needed_for_next_round']}", True, (255, 255, 255))
            screen.blit(progress_text, (bar_x + 10, bar_y - 25))

            # Vidas
            for i in range(MAX_LIVES):
                x = SCREEN_WIDTH - 20 - (i + 1) * 44
                screen.blit(img_heart_full if i < state['lives'] else img_heart_empty, (x, 10))

            # Escudos
            for i in range(MAX_SHIELDS):
                x = SCREEN_WIDTH - 20 - (i + 1) * 44
                screen.blit(img_shield_full if i < state['shields'] else img_shield_empty, (x, 54))

            # Balas
            for i in range(MAX_BULLETS):
                x = SCREEN_WIDTH - 20 - (i + 1) * 44
                screen.blit(img_bullet_full if i < state['bullets'] else img_bullet_empty, (x, 98))

            # Cooldowns
            cd_y = 100
            if state['shield_cooldown'] > 0:
                pct = state['shield_cooldown'] / SHIELD_COOLDOWN_FRAMES
                pygame.draw.rect(screen, (50, 50, 50), (50, cd_y + 30, 200, 20))
                pygame.draw.rect(screen, (255, 0, 0), (50, cd_y + 30, int(200 * pct), 20))
                screen.blit(font.render(f"ESCUDO: {state['shield_cooldown']//30}s", True, (255, 0, 0)), (50, cd_y))
                cd_y += 60

            if state['attack_cooldown'] > 0:
                pct = state['attack_cooldown'] / ATTACK_COOLDOWN_FRAMES
                pygame.draw.rect(screen, (50, 50, 50), (50, cd_y + 30, 200, 20))
                pygame.draw.rect(screen, (255, 100, 0), (50, cd_y + 30, int(200 * pct), 20))
                screen.blit(font.render(f"ATAQUE: {state['attack_cooldown']//30}s", True, (255, 100, 0)), (50, cd_y))

            # Game Over
            if state['game_over']:
                go_font = pygame.font.SysFont('Arial', 64)
                go_surf = go_font.render("GAME OVER", True, (255, 0, 0))
                screen.blit(go_surf, ((SCREEN_WIDTH - go_surf.get_width()) // 2, SCREEN_HEIGHT // 2 - 40))
                hint = font.render("Presiona R para reiniciar", True, COLOR_WHITE)
                screen.blit(hint, ((SCREEN_WIDTH - hint.get_width()) // 2, SCREEN_HEIGHT // 2 + 30))

            # Mostrar pantalla de ronda completada (sincronizada con la medalla)
            if state['medal_timer'] > 0:
                draw_round_complete_screen(screen, SCREEN_WIDTH, SCREEN_HEIGHT, state['current_round'] - 1, state['score'], state['points_needed_for_next_round'])

            # Mostrar medalla animada mientras medal_timer > 0
            if state['medal_timer'] > 0 and img_medals:
                try:
                    base_medal = img_medals[state.get('current_medal_index', 0)]

                    # Pulso MUY suave para no pixelar
                    pulse = 1.0 + 0.06 * math.sin(time.time() * 6)
                    mw, mh = base_medal.get_size()
                    new_w = int(mw * pulse)
                    new_h = int(mh * pulse)

                    mscaled = pygame.transform.smoothscale(base_medal, (new_w, new_h))
                    target_x = SCREEN_WIDTH // 2
                    target_y = SCREEN_HEIGHT - 120  # ajusta este valor a donde la quieras

                    mrect = mscaled.get_rect(center=(target_x, target_y))

                    # Halo para que destaque sobre el vídeo
                    halo_radius = max(new_w, new_h) // 2 + 12
                    halo_surf = pygame.Surface((halo_radius * 2, halo_radius * 2), pygame.SRCALPHA)
                    pygame.draw.circle(halo_surf, (255, 255, 0, 100), (halo_radius, halo_radius), halo_radius)
                    screen.blit(halo_surf, (mrect.centerx - halo_radius, mrect.centery - halo_radius))

                    screen.blit(mscaled, mrect)
                except Exception:
                    pass

                state['medal_timer'] -= 1

            # Eventos
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        running = False
                    elif event.key == pygame.K_r and state['game_over']:
                        state['enemies'] = []
                        state['powerups'] = []
                        state['score'] = 0
                        state['lives'] = MAX_LIVES
                        state['shields'] = MAX_SHIELDS
                        state['bullets'] = MAX_BULLETS
                        state['game_over'] = False
                        state['energy_charge'] = 0
                        state['shield_active'] = False
                        state['shield_cooldown'] = 0
                        state['attack_cooldown'] = 0
                        state['spawn_timer'] = 0
                        state['spawn_pause'] = 0
                        state['powerup_timer'] = 0
                        state['powerup_rate'] = random.randint(POWERUP_SPAWN_MIN, POWERUP_SPAWN_MAX)
                        state['explosion_effect'] = 0
                        state['cd_tick_timer'] = 0
                        state['cd_channel'] = None
                        state['current_round'] = 1
                        state['current_medal_index'] = 0
                        state['points_for_this_round'] = 0
                        state['points_needed_for_next_round'] = ROUND_POINTS_REQUIREMENT
                        state['medal_timer'] = 0
                        state['round_sound_channel'] = None


            pygame.display.flip()
            clock.tick(FPS)
    
    cap.release()
    pygame.quit()
    print("\nJuego terminado")


# Ejecutar el juego
if __name__ == '__main__':
    start_game()


INICIANDO NEON CASTER...

Spritesheet de explosión existe
Cargando assets...
   8 enemigos
   6 frames de explosión
   Sonidos cargados
Assets cargados


Juego terminado
